In [3]:
import json
import pymongo
from flask import Flask, request, jsonify
from flask_mongoengine import MongoEngine

xxyzx = None
app = Flask( __name__ )
app.config[ 'MONGODB_SETTINGS' ] = { 'db': 'tfm00', 'host': 'localhost', 'port': 27017 }
db = MongoEngine()
db.init_app( app )


In [4]:
class User( db.Document ):
	_id = db.StringField()
	gene_symbol = db.StringField()
	
	def to_json( self ):
		return { "_id": self._id, "email": self.gene_symbol }


In [5]:
def retrieve_with_mongoengine( name ):
	user = User.objects( _id=name )
	return jsonify( user.to_json() )


myclient = pymongo.MongoClient( 'mongodb://localhost:27017/' )
mydb = myclient[ 'tfm00' ]
mycol = mydb[ 'mirtarbase' ]


def retrieve_with_pymongo( conditions, fields ):
	myres = mycol.find( conditions, fields )
	return myres


In [6]:
@app.route( '/', methods=[ 'GET' ] )
def mainmenu():
	return jsonify( {
		'help': 'please use one of the shown formats',
		'fmts': {
		'/genes?mirna=<id>': 'get genes regulted by mirna',
		'/mirnas?gene=<id>': 'get mirnas regulting a gene',
		}
	} )


### 1. Qué genes están regulados por un determinado mirna.
### recibe un `_id` y devuelve un `gene_symbol`
@app.route( '/genes', methods=[ 'GET' ] )
def genes():
	name = request.args.get( 'mirna' )
	try:
		res = retrieve_with_mongoengine( name )
	except:
		res = retrieve_with_pymongo( { '_id': name }, { '_id': 0, 'gene_symbol': 1 } )
		res = next( res, { 'gene_symbol': None } )
	return jsonify( res )


### 2. Qué mirnas regulan determinados genes.
### recibe un `gene_symbol` y se devuelve uno o varios `_id`s
@app.route( '/mirnas', methods=[ 'GET' ] )
def mirnas():
	name = request.args.get( 'gen' )
	print( name )
	return jsonify( { 'name': name } )


In [12]:
res = retrieve_with_pymongo(
	{
	'_id': 'MIRT002091',
	},
	{
	'_id': 0,
	'gene_symbol': 1,
	},
)
res = next( res, { 'gene_symbol': None } )
res, type( res )


({'gene_symbol': 'CSD2'}, dict)

In [17]:
from gprofiler import GProfiler

gp = GProfiler( return_dataframe=False )
res = gp.profile(
	organism='hsapiens',
	query=[ 'DNM1L' ],
)
for r in res:
	yield r[ 'native' ]


[{'description': '"A vesicle derived via budding from a mitochondrion. These vesicles often contain inner membrane and, much more rarely, cristae." [GOC:bc, GOC:pad, GOC:PARL-UCL, PMID:18207745, PMID:20619655, PMID:22226745, PMID:23300790]',
  'effective_domain_size': 22122,
  'intersection_size': 1,
  'name': 'mitochondrion-derived vesicle',
  'native': 'GO:0099073',
  'p_value': 0.029870398315825383,
  'parents': ['GO:0031410'],
  'precision': 1.0,
  'query': 'query_1',
  'query_size': 1,
  'recall': 0.3333333333333333,
  'significant': True,
  'source': 'GO:CC',
  'term_size': 3},
 {'description': 'DNM1L-FIS1 complex',
  'effective_domain_size': 3383,
  'intersection_size': 1,
  'name': 'DNM1L-FIS1 complex',
  'native': 'CORUM:4200',
  'p_value': 0.04993169344335535,
  'parents': ['CORUM:0000000'],
  'precision': 1.0,
  'query': 'query_1',
  'query_size': 1,
  'recall': 0.5,
  'significant': True,
  'source': 'CORUM',
  'term_size': 2}]